<a href="https://colab.research.google.com/github/Marcusleeleelee/FTEC4998-4999/blob/main/FTEC4998_4999.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
# Step 0: Import the packages - ok
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from time import sleep
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from google.colab import drive
from tqdm import tqdm
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
# Step 1: Utils - ok
def uni_list(input): return list(set(input))
def perform_pca(df, n_components):

    # Performing PCA
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(df)

    # Creating a DataFrame with the top 15 components
    pca_df = pd.DataFrame(data=principal_components, index=df.index)

    # Retaining the original column names for the new DataFrame
    retained_columns = df.columns[:n_components]
    pca_df.columns = retained_columns

    return pca_df

In [73]:
class Dataset():
    def __init__(self, file_path):
        self.dataset = pd.read_feather(file_path)
        self.X_train, self.y_train = None, None
        self.X_test, self.y_test = None, None
        self.scalers = None
        self.pca = None
        self.label = 'loan_condition_cat'

    def show(self, rows=10):
        return self.dataset.head(rows)

    def basic_processing(self):
        temp_func_1 = lambda x: '<=2009' if str(x) in ['2007', '2008', '2009'] else ("[2010, 2012]" if str(x) in ['2010', '2011', '2012'] else '>=2013')
        columns_to_delete = [
            'id', 'issue_d', 'home_ownership_cat', 'income_category', 'income_cat', 'term_cat', 'application_type_cat',
            'purpose_cat', 'interest_payment_cat', 'loan_condition'
        ]
        self.dataset.drop(columns=columns_to_delete, inplace=True)
        self.dataset['grade'] = self.dataset['grade'].apply(temp_func_1)
        self.dataset['final_d'] = self.dataset['final_d'].apply(lambda x: str(x)[-4:]).apply(temp_func_1)
        self.dataset = pd.get_dummies(self.dataset, columns=['year', 'final_d', 'home_ownership', 'term', 'application_type',
                                                             'purpose', 'interest_payments', 'grade', 'region'], dtype=int)

    def train_test_split(self, test_size=0.2, random_state=42):
        X = self.dataset.drop(columns=[self.label])
        y = self.dataset[self.label]
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    def preprocessing_train(self):
        scaler = StandardScaler()
        self.X_train = pd.DataFrame(scaler.fit_transform(self.X_train), columns=self.X_train.columns)
        self.scalers = scaler

        # Perform PCA
        self.pca = PCA(n_components=30)
        self.X_train = pd.DataFrame(self.pca.fit_transform(self.X_train))

    def preprocessing_test(self):
        # Apply stored scalers
        self.X_test = pd.DataFrame(self.scalers.transform(self.X_test), columns=self.X_test.columns)

        # Apply PCA
        self.X_test = pd.DataFrame(self.pca.transform(self.X_test))

In [74]:
# Calculating # ok
data = Dataset('/content/drive/My Drive/Colab Notebooks/FTEC4998_9/loan_final313_processed.feather')
data.basic_processing()
data.train_test_split()
data.preprocessing_train()
data.preprocessing_test()

In [76]:
# Data conversion # ok
train_x, train_y = data.X_train, data.y_train
test_x, test_y = data.X_test, data.y_test
X_train, y_train = train_x.to_numpy(), train_y.values.ravel()
X_test, y_test = test_x.to_numpy(), test_y.values.ravel()
counts = np.mean(y_train == 1) * 100
print(counts)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
# Ensure y_train is binary
assert set(y_train).issubset({0, 1}), "Target values must be 0 or 1 for binary classification."

# Convert to tensors and move to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)

7.5910370853482805
(709903, 30) (709903,)
(177476, 30) (177476,)


In [30]:
# Train, predict, and accuracy functions
def train_model(model): # ok
    model.train()
    for epoch in range(model.epochs):
        model.optimizer.zero_grad()
        outputs = model(model.X_train_tensor)
        loss = model.criterion(outputs, model.y_train_tensor)
        loss.backward()
        model.optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')


def predict_model(model, X_tensor):
    model.eval()
    with torch.no_grad():
        outputs = model(X_tensor).squeeze()
        return (outputs > 0.5).float()

def calculate_accuracy(model, X_tensor, y_tensor):
    X_tensor = X_tensor.to(next(model.parameters()).device)
    y_tensor = y_tensor.to(next(model.parameters()).device)
    predictions = predict_model(model, X_tensor)

    # Ensure predictions and labels are the same shape
    predictions = predictions.squeeze()
    y_tensor = y_tensor.squeeze()

    correct = (predictions == y_tensor).sum().item()
    accuracy = correct / y_tensor.size(0)
    return accuracy

In [31]:
# Train ANN
# MLP model
class ANN(nn.Module):
    def __init__(self, X_train_tensor, y_train_tensor, lr=0.001):
        super(ANN, self).__init__()
        self.input_dim = X_train_tensor.shape[1]
        self.net = nn.Sequential(
            nn.Linear(self.input_dim, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(0.03),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.epochs = 100
        self.y_train_tensor = y_train_tensor
        self.X_train_tensor = X_train_tensor

    def forward(self, x):
        return self.net(x)
ann_model = ANN(X_train_tensor, y_train_tensor).to(device)
train_model(ann_model)
ann_predictions = predict_model(ann_model, X_train_tensor).cpu().numpy()
print('ANN train:', calculate_accuracy(ann_model, ann_model.X_train_tensor, ann_model.y_train_tensor))
print('ANN test:', calculate_accuracy(ann_model, X_test_tensor, y_test_tensor))

Epoch 0, Loss: 0.6712297797203064
Epoch 10, Loss: 0.567823052406311
Epoch 20, Loss: 0.5015900135040283
Epoch 30, Loss: 0.4576817750930786
Epoch 40, Loss: 0.4214310646057129
Epoch 50, Loss: 0.38850027322769165
Epoch 60, Loss: 0.35972169041633606
Epoch 70, Loss: 0.33442872762680054
Epoch 80, Loss: 0.3118170201778412
Epoch 90, Loss: 0.29266512393951416
ANN: 0.9464997330621225


In [42]:
print(ann_predictions)
print(type(ann_predictions))
print(np.unique(ann_predictions))

[0. 0. 0. ... 0. 0. 0.]
<class 'numpy.ndarray'>
[0. 1.]


In [77]:
# Logistic Regression as a neural network
class LogisticRegressionModel(nn.Module):
    def __init__(self, X_train_tensor, y_train_tensor, lr=0.01):
        super(LogisticRegressionModel, self).__init__()
        self.input_dim = X_train_tensor.shape[1]
        self.net = nn.Sequential(
            nn.Linear(self.input_dim, 1),
            nn.Sigmoid()
        )
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.epochs = 700
        self.y_train_tensor = y_train_tensor
        self.X_train_tensor = X_train_tensor

    def forward(self, x):
        return self.net(x)
log_reg = LogisticRegressionModel(X_train_tensor, y_train_tensor).to(device)
train_model(log_reg)
log_reg_predictions = predict_model(log_reg, X_train_tensor).cpu().numpy()
print('Logistic Regression train:', calculate_accuracy(log_reg, log_reg.X_train_tensor, log_reg.y_train_tensor))
print('Logistic Regression test:', calculate_accuracy(log_reg, X_test_tensor, y_test_tensor))

Epoch 0, Loss: 0.6930169463157654
Epoch 10, Loss: 0.5847083926200867
Epoch 20, Loss: 0.5268315672874451
Epoch 30, Loss: 0.490772545337677
Epoch 40, Loss: 0.4561319351196289
Epoch 50, Loss: 0.4263942539691925
Epoch 60, Loss: 0.4009254574775696
Epoch 70, Loss: 0.37893223762512207
Epoch 80, Loss: 0.3597172498703003
Epoch 90, Loss: 0.342867910861969
Epoch 100, Loss: 0.3280279040336609
Epoch 110, Loss: 0.3149062991142273
Epoch 120, Loss: 0.3032650649547577
Epoch 130, Loss: 0.2929040491580963
Epoch 140, Loss: 0.28365564346313477
Epoch 150, Loss: 0.27537810802459717
Epoch 160, Loss: 0.2679508328437805
Epoch 170, Loss: 0.26127129793167114
Epoch 180, Loss: 0.25525131821632385
Epoch 190, Loss: 0.2498154640197754
Epoch 200, Loss: 0.24489754438400269
Epoch 210, Loss: 0.2404409945011139
Epoch 220, Loss: 0.23639599978923798
Epoch 230, Loss: 0.23271894454956055
Epoch 240, Loss: 0.22937129437923431
Epoch 250, Loss: 0.22632013261318207
Epoch 260, Loss: 0.22353503108024597
Epoch 270, Loss: 0.22099004685

In [78]:
print(log_reg_predictions)
print(type(log_reg_predictions))
print(np.unique(log_reg_predictions))

[0. 0. 0. ... 0. 0. 0.]
<class 'numpy.ndarray'>
[0. 1.]


In [ ]:
class SVMClassifier():
    def __init__(self, X_train, y_train):
        self.pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('svc', SVC())
        ])

        self.param_grid = {
            'svc__C': [0.1],
            'svc__kernel': ['linear'],
            'svc__gamma': ['scale']
        }
        self.X_train = X_train
        self.y_train = y_train
        self.grid_search = None

    def fit(self, cv=2):
        self.grid_search = GridSearchCV(self.pipeline, self.param_grid, cv=cv, verbose=3, n_jobs=-1)
        self.grid_search.fit(self.X_train, self.y_train)
        print("Best Parameters:", self.grid_search.best_params_)

    def predict(self, X_test):
        return self.grid_search.best_estimator_.predict(X_test)

    def calculate_accuracy(self, X, y):
        predictions = self.predict(X)
        accuracy = accuracy_score(y, predictions)
        return accuracy

SVM_model = SVMClassifier(train_x, train_y)
SVM_model.fit()
SVM_predictions = SVM_model.predict(SVM_model.X_train, SVM_model.y_train)
print('SVM: ', SVM_model.calculate_accuracy(SVM_model.X_train, SVM_model.y_train))

Fitting 2 folds for each of 1 candidates, totalling 2 fits


In [ ]:
print(SVM_predictions)
print(type(SVM_predictions))
print(np.unique(SVM_predictions))

In [ ]:
class NaiveBayesClassifier():
    def __init__(self, X_train, y_train):
        self.pipeline = Pipeline([
            ('scaler', StandardScaler()),  # Optional for Naive Bayes
            ('nb', GaussianNB())
        ])
        self.X_train = X_train
        self.y_train = y_train
        self.model = None

    def fit(self):
        self.model = self.pipeline.fit(self.X_train, self.y_train)

    def predict(self, X_test):
        return self.model.predict(X_test)

    def calculate_accuracy(self, X, y):
        predictions = self.predict(X)
        accuracy = accuracy_score(y, predictions)
        return accuracy
NB_model = NaiveBayesClassifier(X_train, y_train)
NB_model.fit()
accuracy = NB_model.calculate_accuracy(NB_model.X_train, NB_model.y_train)
print('Naive Bayes Accuracy:', accuracy)

In [ ]:
# Predict with PyTorch models and move predictions to CPU
svm_predictions = predict_model(svm_model, X_train_tensor).cpu().numpy()
nb_predictions = nb_model.predict(nb_model.X_train_tensor)

In [ ]:
# Model Accuracy:

In [ ]:
# Collect predictions for stacking
predictions = {
    "log_reg": log_reg_predictions,
    "ANN": ann_predictions,
    'svm':svm_predictions,
    'nb':nb_predictions,
}

In [ ]:
print('Log_reg:', calculate_accuracy(log_reg_model, log_reg_model.X_train_tensor, log_reg_model.y_train_tensor))

Log_reg: 0.594665750109522


In [ ]:
print('SVM:', calculate_accuracy(svm_model, svm_model.X_train_tensor, svm_model.y_train_tensor))

SVM: 0.043458049902592326


In [ ]:
print('NB:', calculate_accuracy(nb_model, nb_model.X_train_tensor, nb_model.y_train_tensor))

TypeError: 'NoneType' object is not iterable

In [ ]:
# Stack predictions as features for AdaBoost
stacked_features = np.column_stack(list(predictions.values()))

# Initialize and train AdaBoost
ada_boost = AdaBoostClassifier(n_estimators=30, random_state=29)
ada_boost.fit(stacked_features, y_train)

# Evaluate train
stacked_accuracy = ada_boost.score(stacked_features, y_train)
print("Stacked model accuracy:", stacked_accuracy)